Laburanding en ver de diferenciar una senial de noise de una de FM, para luego extender a varias mas. Version para re ejecutar y dejar el mejor.

Data logged using:
uhd_ -f -b 

**Se ajusto todo para debuggear porque no sale de .5 accuracy, ver problema.**

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dropout, Conv1D, Dense, Activation, MaxPooling1D, Input
from tensorflow.keras.utils import Sequence as Sequence
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
from google.colab import drive
from datetime import datetime
import ast

sample_rate = 100000
signals_count = 2
epochs = 20
batch_size = 128
capture_size = 2048

# Debug
#train_size = 16
#test_size = 4
#capture_size = 8
# Test
#train_size = 50
#test_size = 5
# Real
train_size = 1000
test_size = 200

dir_base_model = "/content/gdrive/My Drive/DeepLearning/sigid/test-"
print(tf.__version__)

In [0]:
drive.mount('/content/gdrive')

# Create a dtype with the binary data format and the desired column names
dt = np.dtype([('i', 'f4'), ('q', 'f4')])
# Load noisy data
data = np.fromfile(dir_base_model + "nothing.iq", dtype=dt)
dfa = pd.DataFrame(data)
dfa["label"] = "BLANK"
npArray = np.array(range(len(dfa)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfa["time"] = npArray / sample_rate
# Load FM data
data = np.fromfile(dir_base_model + "999.iq", dtype=dt)
dfb = pd.DataFrame(data)
dfb["label"] = "FM"
npArray = np.array(range(len(dfb)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfb["time"] = npArray / sample_rate

frames = [dfa, dfb]
result = pd.concat(frames)

In [0]:
train = []
test = []

# Build datasets for train 
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfa[ii:ii + capture_size].values)
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfb[ii:ii + capture_size].values)

# Build datasets for test 
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfa[ii:ii + capture_size].values)
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfb[ii:ii + capture_size].values)

In [0]:
print('Capture size dfa BLANK: ' + str(len(dfa.values)))
print('Capture size dfb FM: ' + str(len(dfb.values)))
#print(dfa[1:capture_size]['i'])

In [0]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfa[1:capture_size]['time'], y=dfa[1:capture_size]['i'], mode='lines', name='i'))
fig.add_trace(go.Scatter(x=dfa[1:capture_size]['time'], y=dfa[1:capture_size]['q'], mode='lines', name='q'))
fig.update_layout(
    title="NOISE",
    xaxis_title="time",
    yaxis_title="value",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=dfb[1:capture_size]['time'], y=dfb[1:capture_size]['i'], mode='lines', name='i'))
fig.add_trace(go.Scatter(x=dfb[1:capture_size]['time'], y=dfb[1:capture_size]['q'], mode='lines', name='q'))
fig.update_layout(
    title="FM",
    xaxis_title="time",
    yaxis_title="value",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [0]:
dfa[1:capture_size]

In [0]:
# Plot some sample data to check iq files
import plotly.express as px
interleave = 2
fig = px.line(dfa[1+train_size * interleave + capture_size:capture_size], x="time", y="i", labels={'X':'I'}, width=1000, height=300)
fig.show()
fig = px.line(dfb[1+train_size * interleave + capture_size:capture_size], x="time", y="i", labels={'X':'I'}, width=1000, height=300)
fig.show()

In [0]:
def get_signal_dataframes(df):
  interleave = 2
  dx = []
  dy = []
  tx = []
  ty = []

  # Generate train dataset 
  dfax = df[['i', 'q']] 
  dfay = df[['label']] 
  dfax = dfax * 10000.0
  for i in range(0, train_size):
    newx = dfax[interleave * i:interleave * i + capture_size]
    dx.append(newx)    
    newy = dfay[interleave * i:interleave * i + capture_size]
    dy.append(newy)

  # Offset to separate train from test datain df
  offset = train_size * interleave + capture_size

  print("Offset " + str(offset))

  # Generate test dataset 
  for i in range(0, test_size):
    newx = dfax[interleave * i + offset:interleave * i + capture_size + offset]
    tx.append(newx)
    newy = dfay[interleave * i + offset:interleave * i + capture_size + offset]
    ty.append(newy)

  # Convert lists to dataframes
  train_x = pd.DataFrame(dx) 
  train_y = pd.DataFrame(dy) 
  test_x = pd.DataFrame(tx) 
  test_y = pd.DataFrame(ty) 
  return train_x, train_y, test_x, test_y

def add_dataframe_for_signal(df, train_x, train_y, test_x, test_y):
  x, y, tx, ty = get_signal_dataframes(df)
  train_x.append(x)
  train_y.append(y)
  test_x.append(tx)
  test_y.append(ty)

train_x = []
train_y = []
test_x = []
test_y = []
add_dataframe_for_signal(dfa, train_x, train_y, test_x, test_y)
add_dataframe_for_signal(dfb, train_x, train_y, test_x, test_y)

# Build dataframes 
train_x = pd.concat(train_x)
train_y = pd.concat(train_y)
test_x = pd.concat(test_x)
test_y = pd.concat(test_y)

# Shuffle data
permutations = np.random.permutation(len(train_y))
train_x = train_x.iloc[permutations]
train_y = train_y.iloc[permutations]
permutations = np.random.permutation(len(test_x))
test_x = test_x.iloc[permutations]
test_y = test_y.iloc[permutations]

In [0]:
def recreate_np_array_x(d, size):
  new = np.ndarray(shape=(signals_count * size, 2, capture_size), dtype=float)
  # train or test size i
  for i in range(len(d.values)):
    # 2048 j
    for j in range(len((d.values)[i])):
        new[i][0][j] = ((d.values)[i])[0].values[j][0]
        new[i][1][j] = ((d.values)[i])[0].values[j][1]
  return new

def recreate_np_array_y(d, size):
  new = np.ndarray(shape=(signals_count * size), dtype=np.int16)
  # 100 i
  for i in range(len(d.values)):
    result = ((d.values)[i])[0].values[0][0]
    if (result == 'FM'):
      new[i] = 1
    else:
      new[i] = 0
  return new

train_x = recreate_np_array_x(train_x, train_size)
train_y = recreate_np_array_y(train_y, train_size)
test_x = recreate_np_array_x(test_x, test_size)
test_y = recreate_np_array_y(test_y, test_size)

In [0]:
# Real model to test: .771 accuracay
def model_original():
  model = Sequential()
  model.add(Conv1D(filters=10, kernel_size=16, strides=1, padding='same', activation='relu', input_shape=(2, capture_size)))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Conv1D(filters=10, kernel_size=12, strides=1, padding='same', activation='relu', input_shape=(6, int(capture_size / 2))))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Conv1D(filters=10, kernel_size=6, strides=1, padding='same', activation='relu', input_shape=(6, int(capture_size / 4))))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Conv1D(filters=10, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(6, int(capture_size / 8))))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

# Real model to test: .805 accuracay
def model_test():
  model = Sequential()
  model.add(Conv1D(filters=10, kernel_size=8, strides=3, padding='same', activation='relu', input_shape=(2, capture_size)))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Dense(20, activation='relu'))
  model.add(Conv1D(filters=10, kernel_size=12, strides=1, padding='same', activation='relu', input_shape=(6, int(capture_size / 2))))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Dense(20, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [0]:
def load_last_model():
  model = tf.keras.models.load_model(dir_base_model + 'best-noise-vs-fm-info-best.model')
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
  return model

In [0]:
# Tensorboard test
from datetime import datetime
logdir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [0]:
# #TODO: Contar los distintos labels de los df
#model = model_original()
model = load_last_model()
model.summary()

# Train
model.fit(x = train_x, y = train_y, batch_size=batch_size, verbose=1, validation_data=(test_x, test_y), epochs=60, callbacks=[tensorboard_callback])

In [0]:
def read_saved_info():
  try:
      file = open(dir_base_model + 'best-noise-vs-fm-info-best.txt', 'rt')
      value = file.read()
      print(value)
      file.close()
      old_evaluation_result = ast.literal_eval(value)
      return old_evaluation_result
  except FileNotFoundError:
      return None

In [0]:
# Evaluate model
evaluation_result = model.evaluate(test_x, test_y, verbose=1)
print(evaluation_result)
print("Current loss is: " + str(evaluation_result))

old_evaluation_result = read_saved_info()
exists = False
print("Old loss is: " + str(old_evaluation_result))
if (old_evaluation_result != None):
  exists = True

if (exists == False or (evaluation_result[1] > old_evaluation_result[1])):
  print("Writting new object becauce of better validation accuracy...")
  file = open(dir_base_model + 'best-noise-vs-fm-info-best.txt', 'wt')
  file.write(str(evaluation_result))
  file.close()
  model.save(dir_base_model + 'best-noise-vs-fm-info-best.model')

In [0]:
# Prepare one line of data to predict
iv = test_x[0][0]
qv = test_x[0][1]
element = 0

td = np.ndarray(shape=(2, 2, capture_size), dtype=float)
for j in range(10):
  for i in range(len(iv)):
    td[0][0][i] = test_x[element+j][0][i]
    td[0][1][i] = test_x[element+j][1][i]

rd = np.ndarray(shape=(2, 2, capture_size), dtype=float)
for j in range(10):
  for i in range(len(iv)):
    rd[0][0][i] = test_y[element+j][0][i]

In [0]:
predictions = model.predict(td)
print(predictions)

In [0]:
# Draw confusion matrix
# https://plot.ly/~francoisp/50/confusion-matrix/#code
